In [13]:
import os # работа с папкой и файлами
import re # регулярные выражения, доп вариант к очистке
import string # работа со строкой

import pandas as pd # работа с таблицей dataframe

import nltk # работа с пакетами языков
from nltk import word_tokenize, ngrams # токенизация и деление на n граммы
from nltk.corpus import stopwords # стопслова, extend

from wordcloud import WordCloud # визуальное отображение
import pymorphy3 # работа с русским языком, pymorphy3

import matplotlib.pyplot as plt # визуальное отображение

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sorokin.m.e\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
# Список файлов в папке
FOLDER_NAME = 'data'

direct_list = os.listdir(FOLDER_NAME)
print(direct_list)

[]


In [15]:
# Имена песен из названия файлов
names = [song.replace(".txt", "") for song in direct_list]
print(names)

[]


In [16]:
# Функция чтения файла
def read_song(filename, folder=FOLDER_NAME):
  text = ''
  with open(f'{folder}/{filename}.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\n', ' ')
  return text

# Чтение данных из папки
songs_list = []

for song in names:
  songs_list.append(read_song(song))

print(songs_list)

[]


In [17]:
# Стоп-слова русского языка
stopwords_list = stopwords.words("russian")

# Собственные стоп-слова
extra_stop = ['твоей', 'свои', 'тобой', 'мой', 'твой', 'мой', 'это']
stopwords_list.extend(extra_stop)

len(stopwords_list)

158